In [55]:
import nfldb as nfl
import itertools as it

import sys
sys.path.append("..")

from nfllib import GameEngine, PlayerStats

In [3]:
engine = GameEngine()

In [4]:
db = nfl.connect()

In [5]:
q = nfl.Query(db)
_ = q.player(full_name="DeAndre Hopkins")
for p in q.as_players():
    print p.player_id

00-0030564


In [7]:
## 2015 season
q = nfl.Query(db)
_ = (q.game(season_year=2015, season_type="Regular")
         .player(player_id="00-0030564")
    )
games = [g.gsis_id for g in q.as_games()]

In [9]:
scores = {
    g: engine.score_game(g, "00-0030564")
    for g in games
}

In [10]:
scores

{'2015091306': {'00-0030564': 27.5},
 '2015092000': {'00-0030564': 7.5},
 '2015092703': {'00-0030564': 20.0},
 '2015100402': {'00-0030564': 19.5},
 '2015100800': {'00-0030564': 21.5},
 '2015101807': {'00-0030564': 31.0},
 '2015102502': {'00-0030564': 8.0},
 '2015110103': {'00-0030564': 19.0},
 '2015111600': {'00-0030564': 13.5},
 '2015112203': {'00-0030564': 25.5},
 '2015112903': {'00-0030564': 5.5},
 '2015120601': {'00-0030564': 16.5},
 '2015121303': {'00-0030564': 6.5},
 '2015122004': {'00-0030564': 13.0},
 '2015122703': {'00-0030564': 20.5},
 '2016010304': {'00-0030564': 11.5}}

In [31]:
## Look at DeAndre Hopkins vs. Opponent. For each opponent, get average
## of all receivers against them

player_id = "00-0030564"
player_team = "HOU"  # not independent but whatever...
player_position = "WR"  # Again, not independent

## Get all games this player was in
q = nfl.Query(db)
_ = (q.game(season_year=2015, season_type="Regular")
         .player(player_id=player_id)
     )
games = [g.gsis_id for g in q.as_games()]

## get player's opponents. Ignore case where player is traded
opponents = []

q = nfl.Query(db)
_ = q.game(team="HOU", season_year=2015, season_type="Regular")
for g in q.as_games():
    t = (g.home_team, g.away_team)
    opponents.append(t[0] if t[0] != player_team else t[1])

In [57]:
## For each opponent, get all opposing opponents, excluding player's team
unique_opponents = set(opponents)
vs_position = {}
for o in unique_opponents:
    vs_position[o] = {}
    q = nfl.Query(db)
    _ = q.game(season_year=2015, season_type="Regular", team=o)
    for g in q.as_games():
        
        vs_position[o][g.gsis_id] = set(
            pp.player_id for pp in 
            it.ifilter(
                lambda x: x.receiving_rec > 0 and x.team != o, 
                g.play_players
            )
        )

In [58]:
vs_position

{'ATL': {'2015091400': {'00-0023564',
   '00-0023921',
   '00-0025549',
   '00-0027758',
   '00-0027864',
   '00-0028009',
   '00-0030061',
   '00-0030990',
   '00-0031299',
   '00-0031549'},
  '2015092004': {'00-0027155',
   '00-0027450',
   '00-0027994',
   '00-0028838',
   '00-0029262',
   '00-0031235',
   '00-0031462',
   '00-0031953'},
  '2015092700': {'00-0022127', '00-0029000', '00-0029004', '00-0030388'},
  '2015100402': {'00-0026796',
   '00-0027718',
   '00-0028052',
   '00-0029438',
   '00-0030564',
   '00-0031302',
   '00-0031600'},
  '2015101103': {'00-0026345',
   '00-0029119',
   '00-0030404',
   '00-0031068',
   '00-0031939',
   '00-0031941'},
  '2015101500': {'00-0022943',
   '00-0027861',
   '00-0027966',
   '00-0030216',
   '00-0030348',
   '00-0030663',
   '00-0031236'},
  '2015102508': {'00-0024268',
   '00-0024389',
   '00-0027651',
   '00-0029708',
   '00-0030521'},
  '2015110105': {'00-0027135',
   '00-0029613',
   '00-0031273',
   '00-0031390',
   '00-0031408',